In [1]:
TRY1 = "fʹαr (fear) « homme », le génitif fʹirʹ (fir)"

In [4]:
FORM_MAPS = {
    "le génitif": "genetive",
    "gen.": "genetive",
    "dat.": "dative",
    "plur.": "plural"
}

In [15]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) », de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\) « ([^»]+) », dans ([^«]+) « ([^»]+) »$"

In [10]:
m = re.match(REGEX_CASE, TRY1)
m.groups()

('fʹαr', 'fear', 'homme', 'le génitif', 'fʹirʹ', 'fir')

In [ ]:
DATA = {}

In [12]:
SEC6 = """
pɑkə (paca) « sac ».	pʹαkə (peaca) « péché ».
bo꞉ (bó) « vache ».	bʲo꞉ (beó) « vivant ».
tᴀᴜn (tonn) « une vague ».	tʲaᴜn (teann) « trapu ».
do꞉ (dóghadh) « brûler ».	dʲo꞉ (deó), dans go dʲo꞉ « toujours ».
kɑrtʹ (cairt) « voiture »	kʹαrt (ceart) « droit, juste »
gɑl (gal) « fumée ».	gʹαl (geal) « brillant ».
mᴜ꞉ⁱnʹ (móin) « tourbe ».	mʲᴜ꞉n (meón) « caractère ».
ə ŋᴀᴜl (i ngabhal) « entre, mêlé à », de gᴀᴜl (gabhal) « fourche ». ɩ ŋʲaᴜl (i ngeall) « en garantie », de gʲaᴜl (geall) « promesse ».
kɑ꞉s (cás) « destin ».	kɑ꞉ʃ (cáis) « fromage ».
fɑd (fad) « longueur ».	fʹαd (fead) « sifflet ».
do vᴜ꞉ⁱnʹ (do mhóin) « ta tourbe ».	do vʲᴜ꞉n (do mheón) « ton content ».
də χɑrtʹ (do chairt) « ta voiture ».	də çαrt (do cheart) « ton droit »
ɑnəǥɑl (anaghal) « fumée épaisse ».	ɑnɩjαl (anagheal) « très brillant ».
kɑ꞉l (cál) « droit, récla­mation ».	kɑ꞉ⁱlʹ (cáil) « répu­tation ».
ᴜ꞉r (úr) « frais ».	ᴜ꞉ⁱrʹ (úir) « terre, terreau ».
"""

In [18]:
for l in SEC6.split("\n"):
    if l != "":
        for p in l.split("."):
            m = re.match(REGEX_BASE, p.strip())
            print(m)

<re.Match object; span=(0, 19), match='pɑkə (paca) « sac »'>
<re.Match object; span=(0, 23), match='pʹαkə (peaca) « péché »'>
None
<re.Match object; span=(0, 18), match='bo꞉ (bó) « vache »'>
<re.Match object; span=(0, 21), match='bʲo꞉ (beó) « vivant »'>
None
<re.Match object; span=(0, 25), match='tᴀᴜn (tonn) « une vague »'>
<re.Match object; span=(0, 23), match='tʲaᴜn (teann) « trapu »'>
None
<re.Match object; span=(0, 24), match='do꞉ (dóghadh) « brûler »'>
None
None
None
<re.Match object; span=(0, 19), match='gɑl (gal) « fumée »'>
<re.Match object; span=(0, 24), match='gʹαl (geal) « brillant »'>
None
<re.Match object; span=(0, 24), match='mᴜ꞉ⁱnʹ (móin) « tourbe »'>
<re.Match object; span=(0, 26), match='mʲᴜ꞉n (meón) « caractère »'>
None
None
None
None
<re.Match object; span=(0, 21), match='kɑ꞉s (cás) « destin »'>
<re.Match object; span=(0, 23), match='kɑ꞉ʃ (cáis) « fromage »'>
None
<re.Match object; span=(0, 22), match='fɑd (fad) « longueur »'>
<re.Match object; span=(0, 23), match='f